<a href="https://colab.research.google.com/github/Thaleslsilva/DataScience/blob/master/Graficos_Espaciais_Mapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplos de Gráficos Espaciais e Mapas

### Pacotes Python Para Manipulação de Dados e Visualização

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Manipulação de Dados
import numpy as np
import pandas as pd

# Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

# Supress warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Thales de Lima Silva" --iversions

## Carregando os Dados

Dados indisponíveis devido ao tamanho dos arquivos. Favor entrar em contato se desejar obter uma cópia: medvet21@gmail.com

In [ ]:
# Carrega os dados
shoot = pd.read_csv('police-violence-in-the-us-shootings_wash_post.csv')
shoot.head()

In [ ]:
# Carrega os dados
terror = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = 'ISO-8859-1')
terror.head()

In [ ]:
# Carrega os dados
brazil = pd.read_csv('brazilian-ecommerce-olist_geolocation_dataset.csv')
brazil.tail()

## Choropleth Maps
**Objetivo**: Exibir mapa de calor por contagem.

Qual o volume de mortes de pessoas negras em todos os estados dos EUA?

In [ ]:
# Prepara os dados
black_state = shoot[shoot['race']=='B']['state'].value_counts().to_frame().reset_index().rename(columns = {'index':'state','state':'count'})

# Figura
fig = go.Figure(go.Choropleth(locations = black_state['state'],
                              z = black_state['count'].astype(float),
                              locationmode = 'USA-states',
                              colorscale = 'Reds',
                              autocolorscale = False,
                              text = black_state['state'], 
                              marker_line_color = 'white', 
                              colorbar_title = "Milhões USD",
                              showscale = False,))

# Layout
fig.update_layout(title_text = 'Mortes de Negros Por Estado (2015-2020)',
                  title_x = 0.5,
                  geo = dict(scope = 'usa',
                             projection = go.layout.geo.Projection(type = 'albers usa'),
                             showlakes = True, 
                             lakecolor = 'rgb(255, 255, 255)'))

# Template de cores
fig.update_layout(template = "plotly_dark")

# Gráfico
fig.show()

### Mapbox com Linhas
**Objetivo**: Exibir limites e densidade por linhas usando latitude e longitude.

Como as fronteiras do comércio eletrônico são estendidas para três estados do Brasil - AC, MA e AP?

In [ ]:
# Prepara os dados
states = brazil[(brazil['geolocation_state']=='AC')|(brazil['geolocation_state']=='MA')|(brazil['geolocation_state']=='AP')]

# Figura
fig = px.line_mapbox(states, 
                     lat = "geolocation_lat", 
                     lon = "geolocation_lng", 
                     color = "geolocation_state", 
                     zoom = 3, 
                     height = 300)

# Layout
fig.update_layout(mapbox_style = "stamen-terrain", 
                  mapbox_zoom = 2,
                  mapbox_center_lat = -11,
                  margin = {"r":0,"t":0,"l":0,"b":0})

# Gráfico
fig.show()

### Bubble Maps
**Objetivo**: Exibir densidade de valores com bolhas sobre latitude e longitude.

Quantas pessoas foram mortas em ataques terroristas em cada estado dos EUA?

In [ ]:
# Prepara os dados
us_terror = terror.loc[(terror['country_txt']=='United States') & (terror['provstate']!='Unknown')][['provstate','latitude','longitude','nkill']]
map_terror = us_terror.groupby(['provstate']).agg({'nkill':'sum', 'latitude':'mean','longitude':'mean'}).reset_index()

# Parâmetros para os gráficos
limits = [(0,5), (6,15), (16,3000)]
colors = ["royalblue", "crimson", "lightseagreen"]
cities = []
scale = 1

# Figura
fig = go.Figure()

# Loop para configuração dos parâmetros associados aos dados
for i in range(len(limits)):
    lim = limits[i]
    df_sub = map_terror[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(locationmode = 'USA-states',
                                lon = df_sub['longitude'],
                                lat = df_sub['latitude'],
                                marker = dict(size = df_sub['nkill'] / scale,
                                              color = colors[i],
                                              line_color = 'rgb(40,40,40)',
                                              line_width = 0.5,
                                              sizemode = 'area'),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(title_text = 'Ataques Terroristas nos EUA',
                  title_x = 0.5,
                  showlegend = True,
                  geo = dict(scope = 'usa',
                             landcolor = 'rgb(217, 217, 217)'))

# Gráfico
fig.show()

### Mapbox Density
**Objetivo**: Exibir densidade de valores com mapa de calor sobre latitude e longitude.

Quantas pessoas foram mortas em ataques terroristas em todo o mundo?

In [ ]:
# Prepara os dados
all_terror = terror[['city','latitude','longitude','nkill']]
map_all_terror  =all_terror.groupby(['latitude','longitude']).agg({'nkill':'sum'}).reset_index()

# Mapbox
fig = px.density_mapbox(map_all_terror, 
                        lat = 'latitude', 
                        lon = 'longitude', 
                        z = 'nkill', 
                        radius = 10,
                        center = dict(lat = 31, lon = 36), zoom = 1, mapbox_style = "stamen-terrain")

# Layout
fig.update_layout(title_text = 'Pessoas Mortas em Ataques Terroristas ao Redor do Mundo',
                  title_x = 0.5, 
                  showlegend = True)

# Gráfico
fig.show()

### Mapbox Layers
**Objetivo**: Exibir pontos de latitude e longitude.

Qual o total de lojas nos estados RO, AM, AC, AP, RR no Brasil?

In [ ]:
# Prepara os dados
brazil_5 = brazil[(brazil['geolocation_state']=='RO')|(brazil['geolocation_state']=='AM')|(brazil['geolocation_state']=='AC')|(brazil['geolocation_state']=='AP')|(brazil['geolocation_state']=='RR')]

# Figura
fig = px.scatter_mapbox(brazil_5, 
                        lat = "geolocation_lat", 
                        lon = "geolocation_lng", 
                        hover_name = "geolocation_city", 
                        hover_data = ["geolocation_state"],
                        color_discrete_sequence = ["fuchsia"], 
                        zoom = 2, 
                        center = dict(lat = -18, lon = -52), 
                        height = 300)

# Layout
fig.update_layout(mapbox_style = "open-street-map")
fig.update_layout(margin = {"r":0,"t":0,"l":0,"b":0})

# Gráfico
fig.show()

# Fim